In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import sagemaker

# Read dataset

In [ ]:
#%%sh
#wget -N -P data http://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
#unzip data/bank.zip -d data

In [ ]:
DATA_PATH = 'data/'
df = pd.read_csv(DATA_PATH+'bank-full.csv', sep=';')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
enc = OrdinalEncoder()
enc.fit(df)

data = enc.transform(df)

# Split dataset

In [ ]:
train, test = train_test_split(data, test_size=0.2)

In [ ]:
print('all:  ', len(data))
print('train:', len(train))
print('test: ', len(test))

In [ ]:
train_file = DATA_PATH+'bank-train.csv'
pd.DataFrame.from_records(train).to_csv(train_file, index=False, header=True, sep=',')

test_file = DATA_PATH+'bank-test.csv'
pd.DataFrame.from_records(test).to_csv(test_file, index=False, header=True, sep=',')

# Upload to S3

In [ ]:
session = sagemaker.Session()
uri = session.upload_data(path=train_file, key_prefix='bank')
print(uri)